# Table S07. Susceptibility-associated intervals detected by the Inbred

GWAS algorithm

In [ ]:
library(dplyr)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

In [ ]:

# load the functions to save the output
source("bin/gsheets.R")



Attaching package: 'googlesheets4'

The following objects are masked from 'package:googledrive':

    request_generate, request_make


Attaching package: 'kableExtra'

The following objects are masked from 'package:flextable':

    as_image, footnote

The following object is masked from 'package:dplyr':

    group_rows

here() starts at /Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript

$strain_table
$strain_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.html"

$strain_table$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.tsv.zip"

$strain_table$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.docx"

$strain_table$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.csv"


$tox_table_ft
$tox_table_ft$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.html"

$tox_table_ft$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.tsv.zip"

$tox_table_ft$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.docx"

$tox_table_ft$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.csv"


$trait_table
$trait_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S3/table_S3.html"

$trait_table$tsv
[1] "

# Inputs

In [ ]:
# filter to toxins
controls <- c("length_DMSO", "length_Water")

# path to the nemascan output directory
ns_folder <- "data/processed/20231116_Analysis_NemaScan"

# Read in the QTL data
inbred_peaks_df <-
  glue::glue("{ns_folder}/INBRED/Mapping/Processed/QTL_peaks_inbred.tsv") %>%
  data.table::fread()

# Load the tox data
tox <- data.table::fread("data/processed/tox_data/con_metadata.csv") %>%
  dplyr::filter(type == "Toxicant") %>%
  dplyr::select(trait, nice_drug_label2) %>%
  # adjust the uM to µM
  dplyr::mutate(nice_drug_label2 = dplyr::case_when(
    nice_drug_label2 == "uM" ~ "µM",
    TRUE ~ nice_drug_label2
  ))

# load the mappings
all.mappings <- list.files(
  path = glue::glue("{ns_folder}"),
  pattern = "AGGREGATE_mapping",
  recursive = T,
  full.names = T
)

# read in all the mappings
mapping.results <- purrr::map(all.mappings, combine.mappings) %>%
  Reduce(rbind, .)


# Process the data

In [ ]:
inbred_mappings_df <- mapping.results %>%
  dplyr::select(marker, trait, log10p, BETA, AF1, var.exp, algorithm) %>%
  dplyr::mutate(var_type = stringr::str_extract(trait, pattern = "length|CV_length")) %>%
  dplyr::filter(!trait %in% controls) %>%
  dplyr::filter(algorithm == "Inbred" & var_type == "length") %>%
  dplyr::distinct() %>%
  dplyr::left_join(., tox, by = "trait") %>%
  # select the columns we want before joining with the peaks
  dplyr::select(
    marker,
    trait,
    nice_drug_label2,
    BETA,
    AF1,
    var.exp
  )

# Process inbred data
inbred_clean <- inbred_mappings_df %>%
  dplyr::left_join(inbred_peaks_df, by = c("trait", "marker")) %>%
  dplyr::mutate(
    interval = paste0(CHROM, ":", startPOS, "-", endPOS)
  ) %>%
  dplyr::select(
    condition = nice_drug_label2,
    marker,
    interval,
    log10p,
    BETA,
    AF1,
    var.exp
  ) %>%
  dplyr::arrange(condition, desc(log10p))


# Create flextable

In [ ]:
# Create grouped data by condition
grouped_data <- as_grouped_data(
  x = inbred_clean,
  groups = "condition"
)

# Create flextable from grouped data
qtl_detected_ft <- as_flextable(
  grouped_data,
  col_keys = c(
    "marker",
    "interval",
    "log10p",
    "BETA",
    "AF1",
    "var.exp"
  ),
  hide_grouplabel = TRUE
) %>%
  flextable::set_header_labels(
    values = list(
      "marker" = "Peak Marker",
      "interval" = "Interval",
      "log10p" = "-log10(p-value)",
      "BETA" = "Effect Size",
      "AF1" = "Frequency",
      "var.exp" = "Variance Explained"
    )
  ) %>%
  # make only the groups and header bold
  flextable::bold(
    j = 1,
    i = ~ is.na(marker),
    bold = TRUE,
    part = "body"
  ) %>%
  flextable::bold(part = "header") %>%
  flextable::line_spacing(space = 1, part = "all") %>%
  flextable::align(j = 1, align = "center") %>%
  flextable::align(j = 2, align = "left") %>%
  flextable::align(j = 3, align = "center") %>%
  flextable::align(j = 4, align = "center") %>%
  flextable::align(j = 5, align = "center") %>%
  flextable::align(j = 6, align = "center") %>%
  flextable::align(j = 1, i = ~ is.na(marker), align = "left", part = "body") %>%
  flextable::hline(
    i = ~ is.na(marker),
    part = "body"
  ) %>%
  flextable::set_table_properties(
    layout = "autofit",
    width = 1,
    opts_word = list(
      split = FALSE,
      keep_with_next = TRUE
    )
  )


# Save tables

In [ ]:
# Rename columns to match flextable headers for CSV output
inbred_clean_csv <- inbred_clean %>%
  dplyr::rename(
    "Condition" = condition,
    "Peak Marker" = marker,
    "Interval" = interval,
    "-log10(p-value)" = log10p,
    "Effect Size" = BETA,
    "Frequency" = AF1,
    "Variance Explained" = var.exp
  )

# Save the data table and flextable using standardized functions
save_supp_table_csv(inbred_clean_csv, "qtl_detected_table")
save_supp_table_flextable(qtl_detected_ft, "qtl_detected_table")


In [ ]:

qtl_detected_ft


Peak Marker Interval -log10(p-value) Effect Size Frequency Variance Explained 2,4-D V:14453636 V:13865698-15025162 5.56 20.91 0.07 0.07 Aldicarb III:8311919 III:7790159-8989539 6.19 -50.92 0.05 0.12 IV:2368192 IV:2179845-2507551 5.01 -43.80 0.06 0.09 X:13252751 X:13002956-13505962 4.50 22.70 0.23 0.08 IV:16631628 IV:16474675-17180844 4.42 -19.07 0.45 0.09 IV:13208614 IV:12966415-13439670 4.38 -31.78 0.10 0.08 Arsenic III:1642838 III:28947-2527992 5.87 -18.18 0.13 0.12 II:13692527 II:13521931-14347284 5.34 -17.63 0.13 0.11 Atrazine II:12658388 II:12471656-12749733 4.27 -13.18 0.09 0.08 Cadmium X:5955567 X:5111074-7424675 4.27 18.26 0.31 0.15 Carbaryl I:2596756 I:2394565-2987265 4.31 -11.04 0.48 0.06 Carboxin III:2527992 III:2330097-2677403 5.91 -15.72 0.45 0.10 Chlorothalonil II:5392976 II:4501686-12735930 7.94 -10.90 0.22 0.15 II:13921653 II:13521931-14060605 6.13 -9.32 0.24 0.12 III:4478684 III:3019479-7505522 5.21 -7.74 0.34 0.09 I:1741155 I:1503347-1973889 4.65 -7.57 0.29 0.09 III:1455164 III:1286570-1603611 4.49 -7.69 0.26 0.08 III:2605810 III:1997851-2735727 4.32 -7.24 0.30 0.07 Mancozeb II:10305644 II:9770391-10735023 4.19 10.74 0.06 0.05 Mercury III:2461452 III:2007984-2632011 6.20 -34.52 0.13 0.11 II:626831 II:7007-910200 5.42 -34.68 0.11 0.11 II:3977000 II:3740025-4194656 5.18 -33.91 0.11 0.10 I:2994073 I:2643819-3157610 5.13 -46.22 0.06 0.11 III:3716201 III:3103196-5563041 4.83 -32.69 0.11 0.10 III:10133612 III:9741903-10475212 4.59 41.92 0.06 0.09 V:15476283 V:15076312-15922559 4.35 -44.40 0.05 0.09 II:2730209 II:2628141-2835866 4.16 -39.82 0.06 0.08 Nickel V:1486141 V:1077258-1844647 4.78 -28.11 0.20 0.21 II:12203935 II:12072541-12341169 4.70 -40.23 0.07 0.18 Paraquat 62.5 uM I:3282224 I:3080922-3567564 5.15 29.21 0.06 0.08 X:2848181 X:2674396-4285086 4.55 21.71 0.10 0.08 II:13675568 II:13499019-13782678 4.53 17.40 0.16 0.11 I:11258815 I:8368174-11701489 4.33 26.74 0.06 0.10 Propoxur V:5315004 V:4897242-5628662 5.22 8.26 0.12 0.06 Silver 7.8 uM II:11883855 II:11688376-12058655 4.80 9.22 0.09 0.09 I:1622740 I:1336302-1887214 4.74 5.35 0.44 0.09 II:3016298 II:2622088-3571413 4.69 7.52 0.14 0.09 TPhP 6.25 uM V:7623783 V:6022066-16602903 7.42 -17.21 0.09 0.16 IV:16034033 IV:15887403-16272774 5.64 -19.11 0.05 0.12 Zinc I:4884561 I:3619760-8718857 5.88 -10.65 0.10 0.12